# Comparison of coppertop with regular Python

The following code snippets solve the M&M problem from Chapter 1 in Allen B. Downey's book _Think Bayes_ - a copy of which can be found [here](https://github.com/coppertop-bones/coppertop-libs/blob/main/jupyter/think%20bayes/thinkbayes.pdf). 

See [coppertop-bones/README.md](https://github.com/coppertop-bones/coppertop) for a brief introduction to coppertop piping syntax.

<br>

## Introduction



### The M&M Problem

M&M’s are small candy-coated chocolates that come in a variety of colors.
Mars, Inc., which makes M&M’s, changes the mixture of colors from time
to time.

In 1995, they introduced blue M&M’s. Before then, the color mix in a bag
of plain M&M’s was 30% Brown, 20% Yellow, 20% Red, 10% Green, 10%
Orange, 10% Tan. Afterward it was 24% Blue , 20% Green, 16% Orange,
14% Yellow, 13% Red, 13% Brown.

Suppose a friend of mine has two bags of M&M’s, and he tells me that one
is from 1994 and one from 1996. He won’t tell me which is which, but he
gives me one M&M from each bag. One is yellow and one is green. What is
the probability that the yellow one came from the 1994 bag?

<br>

### A Bayes Refresher

See _["An Essay towards solving a Problem in the Doctrine of Chances"](https://github.com/coppertop-bones/coppertop-libs/blob/main/jupyter/think%20bayes/article.pdf)_.

from PROP 3

$$
\begin{align}
\mathbf{P}\left(B \cap A\right) = \mathbf{P}\left(B\mathbin{\vert}A\right)\cdot \mathbf{P}\left(A\right)\\
\end{align}
$$

and
$$\mathbf{P}(A \cap B) = \mathbf{P}(B \cap A)$$

we have
$$
\begin{align}
\mathbf{P}( A\mathbin{\vert}B) \cdot \mathbf{P}(B)=\mathbf{P}(B\mathbin{\vert}A)\cdot \mathbf{P}(A)
\end{align}
$$

aka
$$
\begin{align}
\mathbf{P}( hypothesis\mathbin{\vert}data) \cdot \mathbf{P}(data)=\mathbf{P}(data\mathbin{\vert}hypothesis)\cdot \mathbf{P}(hypothesis)
\end{align}
$$

<br>

Equivalently (after new data is known) we have the comtemporaneous form, i.e. 

$$
\begin{align}
posterior =prior\cdot likelihood \cdot constant
\end{align}
$$

<br>

## Regular Python version

We need the ability to multiple two discrete functions together, a PMF and a likelihood, and normalise the result into a PMF.

<br>

our _Think Bayes_ style library functions:

In [1]:
def pmfMul(A, B):
    res = {}
    for k in A.keys():
        res[k] = A[k] * B[k]
    return res

def normalise(x):
    t = 0
    for p in x.values():
        t += p
    t = 1 / t
    for k in x.keys():
        x[k] *= t
    return x

<br>

our problem solving script:

In [2]:
bag1994 = dict(Brown=30, Yellow=20, Red=20, Green=10, Orange=10, Tan=10)
bag1996 = dict(Brown=13, Yellow=14, Red=13, Green=20, Orange=16, Blue=24)

for e in [bag1994, bag1996]:
    print(e)
    
prior = normalise(dict(hypA=0.5, hypB=0.5))

likelihood = dict(
    hypA=bag1994['Yellow'] * bag1996['Green'],   # hypA -> yellow is from 1994, green is from 1996
    hypB=bag1994['Green'] * bag1996['Yellow']    # hypB -> green is from 1994, yellow is from 1996
)

posterior = normalise(pmfMul(prior, likelihood))

print(prior)
print(likelihood)
print(posterior)

{'Brown': 30, 'Yellow': 20, 'Red': 20, 'Green': 10, 'Orange': 10, 'Tan': 10}
{'Brown': 13, 'Yellow': 14, 'Red': 13, 'Green': 20, 'Orange': 16, 'Blue': 24}
{'hypA': 0.5, 'hypB': 0.5}
{'hypA': 400, 'hypB': 140}
{'hypA': 0.7407407407407408, 'hypB': 0.25925925925925924}


<br>

### Regular Python version - running several priors with clearer output

<br>

adding some library functions:

In [3]:
def formatDF(s, name, valuesFormat, sep):
    def formatKv(kv):
        k, v = kv
        return f'{k}: {format(v, valuesFormat)}'
    kvStrings = [formatKv(kv) for kv in s.items()]
    return f'{name}({sep.join(kvStrings)})'

def ppMMs(bag):
    kvs = [f'{k}={v}' for k, v in bag.items()]
    print(f'MMs({", ".join(kvs)})')
    return bag

def ppPMF(pmf):
    print(formatDF(pmf, 'PMF: ', '.3f', ', '))
    return pmf

def ppL(l):
    print(formatDF(l, 'L:   ', '.1f', ', '))
    return l

<br>

our problem solving script becomes:

In [4]:
print("\ncase 1")
prior1 = ppPMF(normalise(dict(hypA=0.25, hypB=0.75)))
ppL(likelihood)
ppPMF(normalise(pmfMul(prior1, likelihood)))

print("\ncase 2")
prior2 = ppPMF(normalise(dict(hypA=0.5, hypB=0.5)))
ppL(likelihood)
ppPMF(normalise(pmfMul(prior2, likelihood)))

print("\ncase 3")
prior3 = ppPMF(normalise(dict(hypA=0.75, hypB=0.25)))
ppL(likelihood)
ppPMF(normalise(pmfMul(prior3, likelihood)));


case 1
PMF: (hypA: 0.250, hypB: 0.750)
L:   (hypA: 400.0, hypB: 140.0)
PMF: (hypA: 0.488, hypB: 0.512)

case 2
PMF: (hypA: 0.500, hypB: 0.500)
L:   (hypA: 400.0, hypB: 140.0)
PMF: (hypA: 0.741, hypB: 0.259)

case 3
PMF: (hypA: 0.750, hypB: 0.250)
L:   (hypA: 400.0, hypB: 140.0)
PMF: (hypA: 0.896, hypB: 0.104)


<br>

or maybe:

In [5]:
print("\ncase 1")
ppPMF(normalise(pmfMul(ppPMF(normalise(dict(hypA=0.25, hypB=0.75))), ppL(likelihood))))

print("\ncase 2")
ppPMF(normalise(pmfMul(ppPMF(normalise(dict(hypA=0.50, hypB=0.50))), ppL(likelihood))))

print("\ncase 3")
ppPMF(normalise(pmfMul(ppPMF(normalise(dict(hypA=0.75, hypB=0.50))), ppL(likelihood))));


case 1
PMF: (hypA: 0.250, hypB: 0.750)
L:   (hypA: 400.0, hypB: 140.0)
PMF: (hypA: 0.488, hypB: 0.512)

case 2
PMF: (hypA: 0.500, hypB: 0.500)
L:   (hypA: 400.0, hypB: 140.0)
PMF: (hypA: 0.741, hypB: 0.259)

case 3
PMF: (hypA: 0.600, hypB: 0.400)
L:   (hypA: 400.0, hypB: 140.0)
PMF: (hypA: 0.811, hypB: 0.189)


<br>

## What's the problem coppertop is trying to solve?

In a nutshell coppertop is an attempt to make code easier to read, write and be more reliable.

Our Python code looks reasonably simple and thus far it is. However, once we want to start doing more, experience shows that the code can get messy and hard to understand. As an example compare the regular Python code we developed above with the coppertop version:

<br>

**Regular Python**


```python
bag1994 = ppMMs(dict(Brown=30, Yellow=20, Red=20, Green=10, Orange=10, Tan=10))
bag1996 = ppMMs(dict(Brown=13, Yellow=14, Red=13, Green=20, Orange=16, Blue=24))

likelihood = dict(
    hypA=bag1994['Yellow'] * bag1996['Green'],   # hypA -> yellow is from 1994, green is from 1996
    hypB=bag1994['Green'] * bag1996['Yellow']    # hypB -> green is from 1994, yellow is from 1996
)

print("\ncase 1")
ppPMF(normalise(pmfMul(ppPMF(normalise(dict(hypA=0.25, hypB=0.75))), ppL(likelihood))))

print("\ncase 2")
ppPMF(normalise(pmfMul(ppPMF(normalise(dict(hypA=0.50, hypB=0.50))), ppL(likelihood))))

print("\ncase 3")
ppPMF(normalise(pmfMul(ppPMF(normalise(dict(hypA=0.75, hypB=0.50))), ppL(likelihood))));
```


<br>

**Coppertop**


```python
bag1994 = MMs(Brown=30, Yellow=20, Red=20, Green=10, Orange=10, Tan=10) >> PP
bag1996 = MMs(Brown=13, Yellow=14, Red=13, Green=20, Orange=16, Blue=24) >> PP

likelihood = L(
    hypA=bag1994.Yellow * bag1996.Green,   # hypA -> yellow is from 1994, green is from 1996
    hypB=bag1994.Green * bag1996.Yellow    # hypB -> green is from 1994, yellow is from 1996
)

"\ncase 1" >> PP
PMF(hypA=0.25, hypB=0.75) >> PP >> pmfMul >> (likelihood >> PP) >> normalise >> PP

"\ncase 2" >> PP
PMF(hypA=0.5, hypB=0.5) >> PP >> pmfMul >> (likelihood >> PP) >> normalise >> PP

"\ncase 3" >> PP
PMF(hypA=0.75, hypB=0.25) >> PP >> pmfMul >> (likelihood >> PP) >> normalise >> PP;
```

<br>

For the rest of this notebook, we will show the changes to run the analysis for several priors, in three parts:
1) using >> as a pipe operator to reduce visual function nesting, improve readability of function sequence and reduce the number of parentheses,
2) replace dict with types for PMF, likelihood and the bag of M&Ms,
3) formatting the data structures in a type contextual manner to make the output easier to read.

In the rest of this notebook we will see one way this can be done using coppertop, and, with concision not end up with a ball of spaghetti in the process.

<br>

### Part 1 - pipe operator

import the decorator

In [6]:
from coppertop.pipe import *

<br>

adding piping to our library functions:

In [7]:
@coppertop(style=binary)
def pmfMul(A, B):
    res = {}
    for k in A.keys():
        res[k] = A[k] * B[k]
    return res

@coppertop
def normalise(x):
    t = 0
    for p in x.values():
        t += p
    t = 1 / t
    for k in x.keys():
        x[k] *= t
    return x

@coppertop
def PP(x):
    print(x)
    return x

<br>

our problem solving script:

In [8]:
bag1994 = dict(Brown=30, Yellow=20, Red=20, Green=10, Orange=10, Tan=10) >> PP
bag1996 = dict(Brown=13, Yellow=14, Red=13, Green=20, Orange=16, Blue=24) >> PP

likelihood = dict(
    hypA=bag1994['Yellow'] * bag1996['Green'],  # hypA -> yellow is from 1994, green is from 1996
    hypB=bag1994['Green'] * bag1996['Yellow']   # hypB -> green is from 1994, yellow is from 1996
)

dict(hypA=0.5, hypB=0.5) >> PP >> normalise >> pmfMul >> (likelihood >> PP) >> normalise >> PP;

{'Brown': 30, 'Yellow': 20, 'Red': 20, 'Green': 10, 'Orange': 10, 'Tan': 10}
{'Brown': 13, 'Yellow': 14, 'Red': 13, 'Green': 20, 'Orange': 16, 'Blue': 24}
{'hypA': 0.5, 'hypB': 0.5}
{'hypA': 400, 'hypB': 140}
{'hypA': 0.7407407407407408, 'hypB': 0.25925925925925924}


<br>

### Part 2 - types
* create types, DF, PMF and L, backed by tvmap (a typed dict)
* create type MMs - a bag of M&Ms - backed by tvstruct (a typed struct)

In [9]:
from bones.ts.metatypes import BTAtom
from dm.core.structs import tvstruct, tvmap

import bones.ts.metatypes
bones.ts.metatypes.REPL_OVERRIDE_MODE = True     # allow immutable state in the metatypes module to be updated in a repl

<br>

adding types to our libraray:

In [10]:
# constructors
def _newPmf(cs, *args, **kwargs):
    pmf = tvmap(cs, *args, **kwargs)
    if len(pmf) > 0:               # ensure values sum to 1.0 if we have data
        t = 0
        for p in pmf.values():
            t += p
        t = 1 / t
        for k in pmf.keys():
            pmf[k] *= t
    return pmf

# create new types
DF = BTAtom.ensure('DF') & tvmap   # discrete function
PMF = BTAtom.ensure('PMF') & DF    # probability mass function
L = BTAtom.ensure('L') & DF        # likelihood - not strictly necessary but useful later
MMs = BTAtom.ensure('MMs')

DF.setConstructor(tvmap)
PMF.setConstructor(_newPmf)
L.setConstructor(DF)
MMs.setConstructor(tvstruct);

<br>

overloading our _Think Bayes_ style library functions to use the new types:

In [11]:
@coppertop(style=binary)
def pmfMul(a:PMF, b:L) -> DF:
    answer = DF()
    for k in a.keys():
        answer[k] = a[k] * b[k]
    return answer

@coppertop
def normalise(df:DF) -> PMF:
    return PMF(df)

<br>

our problem solving script:

In [12]:
bag1994 = MMs(Brown=30, Yellow=20, Red=20, Green=10, Orange=10, Tan=10) >> PP
bag1996 = MMs(Brown=13, Yellow=14, Red=13, Green=20, Orange=16, Blue=24) >> PP

prior = PMF(hypA=0.5, hypB=0.5) >> PP

likelihood = L(
    hypA=bag1994.Yellow * bag1996.Green,   # hypA -> yellow is from 1994, green is from 1996
    hypB=bag1994.Green * bag1996.Yellow    # hypB -> green is from 1994, yellow is from 1996
) >> PP

posterior = prior >> pmfMul >> likelihood >> normalise >> PP

MMs(Brown=30, Yellow=20, Red=20, Green=10, Orange=10, Tan=10)
MMs(Brown=13, Yellow=14, Red=13, Green=20, Orange=16, Blue=24)
{'hypA': 0.5, 'hypB': 0.5}
{'hypA': 400, 'hypB': 140}
{'hypA': 0.7407407407407408, 'hypB': 0.25925925925925924}


<br>

### Part 3 - overloads and partials
* Improve display by overloading the PP function for PMF and L

<br>

overloading our pretty printing library functions to use the new types:

In [13]:
@coppertop
def formatDf(s, name, valuesFormat, sep):
    def formatKv(kv):
        k, v = kv
        return f'{k}: {format(v, valuesFormat)}'
    kvStrings = [formatKv(kv) for kv in s.items()]
    return f'{name}({sep.join(kvStrings)})'

formatPmf = formatDf(_, 'PMF: ', '.3f', ', ')      # specialise formatDf for PMF
formatL = formatDf(_, 'L:   ', '.1f', ', ')        # specialise formatDf for L

@coppertop
def PP(x:L) -> L:
    print(x >> formatL)
    return x

@coppertop
def PP(x:PMF):
    print(x >> formatPmf)
    return x

<br>

our problem solving script:

In [14]:
bag1994 = MMs(Brown=30, Yellow=20, Red=20, Green=10, Orange=10, Tan=10) >> PP
bag1996 = MMs(Brown=13, Yellow=14, Red=13, Green=20, Orange=16, Blue=24) >> PP

prior = PMF(hypA=0.5, hypB=0.5)

likelihood = L(
    hypA=bag1994.Yellow * bag1996.Green, 
    hypB=bag1994.Green * bag1996.Yellow
)

posterior = prior >> PP >> pmfMul >> (likelihood >> PP) >> normalise >> PP

MMs(Brown=30, Yellow=20, Red=20, Green=10, Orange=10, Tan=10)
MMs(Brown=13, Yellow=14, Red=13, Green=20, Orange=16, Blue=24)
PMF: (hypA: 0.500, hypB: 0.500)
L:   (hypA: 400.0, hypB: 140.0)
PMF: (hypA: 0.741, hypB: 0.259)


<br>

### Our type and function signatures

<br>

In [15]:
@coppertop(style=binary)
def collect(xs, f):
    return [f(x) for x in xs]

(prior, likelihood, posterior) >> collect >> (lambda x: str(x >> type)) >> collect >> PP
'' >> PP
(prior, likelihood, posterior) >> collect >> (lambda x: str(x >> typeOf)) >> collect >> PP
'' >> PP
(prior, likelihood, posterior) >> collect >> (lambda x: str(x >> typeOf >> type)) >> collect >> PP
'' >> PP;

<class 'dm._core.structs.tvmap'>
<class 'dm._core.structs.tvmap'>
<class 'dm._core.structs.tvmap'>

dm._core.structs.tvmap&DF&PMF
dm._core.structs.tvmap&DF&L
dm._core.structs.tvmap&DF&PMF

<class 'bones.ts.metatypes.BTIntersection'>
<class 'bones.ts.metatypes.BTIntersection'>
<class 'bones.ts.metatypes.BTIntersection'>



In [16]:
pmfMul >> sig >> collect >> PP
'' >> PP;

(py,py)->py <binary>  :   in scratch.pmfMul
(dm._core.structs.tvmap&DF&PMF,dm._core.structs.tvmap&DF&L)->dm._core.structs.tvmap&DF <binary>  :   in scratch.pmfMul



In [17]:
normalise >> sig >> collect >> PP
'' >> PP;

(py)->py <unary>  :   in scratch.normalise
(dm._core.structs.tvmap&DF)->dm._core.structs.tvmap&DF&PMF <unary>  :   in scratch.normalise



In [18]:
PP >> sig >> collect >> PP;

(py)->py <unary>  :   in scratch.PP
(dm._core.structs.tvmap&DF&L)->dm._core.structs.tvmap&DF&L <unary>  :   in scratch.PP
(dm._core.structs.tvmap&DF&PMF)->py <unary>  :   in scratch.PP


<br>

### Running several priors

<br>

our final problem solving script:

In [19]:
bag1994 = MMs(Brown=30, Yellow=20, Red=20, Green=10, Orange=10, Tan=10)
bag1996 = MMs(Brown=13, Yellow=14, Red=13, Green=20, Orange=16, Blue=24)

likelihood = L(
    hypA=bag1994.Yellow * bag1996.Green, 
    hypB=bag1994.Green * bag1996.Yellow
)

"\ncase 1" >> PP
PMF(hypA=0.25, hypB=0.75) >> PP >> pmfMul >> (likelihood >> PP) >> normalise >> PP

"\ncase 2" >> PP
PMF(hypA=0.5, hypB=0.5) >> PP >> pmfMul >> (likelihood >> PP) >> normalise >> PP

"\ncase 3" >> PP
PMF(hypA=0.75, hypB=0.25)  >> PP >> pmfMul >> (likelihood >> PP) >> normalise >> PP;


case 1
PMF: (hypA: 0.250, hypB: 0.750)
L:   (hypA: 400.0, hypB: 140.0)
PMF: (hypA: 0.488, hypB: 0.512)

case 2
PMF: (hypA: 0.500, hypB: 0.500)
L:   (hypA: 400.0, hypB: 140.0)
PMF: (hypA: 0.741, hypB: 0.259)

case 3
PMF: (hypA: 0.750, hypB: 0.250)
L:   (hypA: 400.0, hypB: 140.0)
PMF: (hypA: 0.896, hypB: 0.104)


<br>

starting the next stage:

In [20]:
@coppertop
def analysis(i_pA):
    i, pA = i_pA
    pB = 1 - pA
    if i: '' >> PP
    f'case {i+1}' >> PP
    return PMF(hypA=pA, hypB=pB) >> PP >> pmfMul >> (likelihood >> PP) >> normalise >> PP

@coppertop
def withIX(xs):
    return enumerate(xs)

results = [0.25, 0.5, 0.75] >> withIX >> collect >> analysis
'' >> PP;

case 1
PMF: (hypA: 0.250, hypB: 0.750)
L:   (hypA: 400.0, hypB: 140.0)
PMF: (hypA: 0.488, hypB: 0.512)

case 2
PMF: (hypA: 0.500, hypB: 0.500)
L:   (hypA: 400.0, hypB: 140.0)
PMF: (hypA: 0.741, hypB: 0.259)

case 3
PMF: (hypA: 0.750, hypB: 0.250)
L:   (hypA: 400.0, hypB: 140.0)
PMF: (hypA: 0.896, hypB: 0.104)



In [21]:
results >> collect >> PP;

PMF: (hypA: 0.488, hypB: 0.512)
PMF: (hypA: 0.741, hypB: 0.259)
PMF: (hypA: 0.896, hypB: 0.104)


<br>

## Summing up

Firstly, we started off with a simple example Python program and suggested things might get messy as we tried to scale it - this is left as an exercise for the reader to more fully.

In a step by step manner, we looked at the @coppertop decorator, piping, adding new types, overloading and partial functions.

Finally, we combined our functions into concise code to analysis the impact of different priors, perhaps hinting at the potential coppertop might offer.